In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board

import numpy as np

2022-03-30 11:03:42.425562: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 11:03:42.425593: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
training_generator = DataLoader(64, "train")
val_generator = DataLoader(64, "val")

In [3]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=2)

In [4]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

h1 = Dense(32, clipped_relu)
h2 = Dense(32, clipped_relu)
out = Dense(1, 'sigmoid')

layerStack = out(h2(h1(layerStack)))

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

2022-03-30 11:03:51.765611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 11:03:51.766183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 11:03:51.766473: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-30 11:03:51.766620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-30 11:03:51.766671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [5]:
model = tf.keras.models.load_model("production/")

In [5]:
#625/625 [==============================] - 76s 120ms/step - loss: 0.0710 - mae: 0.1957 - val_loss: 0.0967 - val_mae: 0.2318
model.fit(training_generator, validation_data=val_generator, epochs=1)

150/156 [===========================>..] - ETA: 0s - loss: 0.0959 - mae: 0.2442

KeyboardInterrupt: 

In [18]:
indicies = get_halfkp_indeicies(Board("r3k1nr/pp2ppbp/n2p2p1/q1p5/3N1P2/1P2P3/PBPP3P/RN1K1B1R w kq - 1 9"))
indicies = indicies.reshape(2, -1)
indicies = (np.array([indicies[0], indicies[0]]), np.array([indicies[1], indicies[1]]))
pred = model.predict(indicies)
print(pred[0])

[0.13856101]


In [19]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

[-0.03286403] -0.5750424331208387 1rbqr1k1/1pp2pp1/p2b3p/P3p3/3pB3/1Q1P2P1/1B2PP1P/R1R3K1 b - - 0 20
[0.42872286] -0.04502410583767502 2k5/7p/3Bp1p1/pb2Pp2/5P2/8/6PP/6K1 b - - 2 35
[0.48281097] 0.9969819732475262 7r/1p6/p1p2kp1/3R4/3P1P2/P3RK2/1P6/8 w - - 0 37
[0.5905397] 0.999369167382091 7r/7p/p1k4p/1p1RR3/1P2B3/7P/6PK/r7 b - - 2 32
[-0.46399802] -0.9995894903236348 2k2bnr/pppr4/8/4q1pp/8/5pP1/PP3P1P/RN3QK1 w - - 0 24
[0.73144317] 1.0 8/8/8/p1K4p/b5kP/3p4/8/8 w - - 0 47
[0.38178885] 0.9546603230906032 r6N/pppnBBpp/2kp4/8/8/2Pb4/P4PPP/2R2RK1 w - - 3 18
[-0.03446245] 0.031181511436509934 rnbqkb1r/pppp1pp1/5n1p/4p3/2P5/2N3P1/PP1PPP1P/R1BQKBNR w KQkq - 1 4
[-0.13979626] -0.2182397668754964 8/p6p/6p1/2pbp1k1/3p2P1/3P2K1/PPP4P/8 w - - 3 33
[-0.28543943] 0.0 8/8/8/8/6kp/8/5K2/8 w - - 0 52
[0.22935808] 0.34864661826708976 r3r1k1/1n4pp/p1Npbp2/8/PP2PP2/2N5/2P1K1PP/R2R4 b - - 0 21
[-0.06159341] 0.8213725114010577 8/R1N3pp/1P1p1k2/2n2p2/2r2P2/5K2/2P3PP/8 w - - 5 34
[0.1743784] 1.0 rnb2rk1/pp3pp

IndexError: index 64 is out of bounds for axis 0 with size 64

In [20]:
import time
# model.save("models/model-{}".format(int(time.time())))
model.save("production/")

INFO:tensorflow:Assets written to: production/assets
